In [1]:
import tensorflow as tf
import keras

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

In [3]:
print("tf version : {}".format(tf.__version__))
print("keras version : {}".format(keras.__version__))
print("numpy version : {}".format(np.__version__))
print("matplotlib version : {}".format(matplotlib.__version__))
print("pandas version : {}".format(pd.__version__))

tf version : 2.6.1
keras version : 2.6.0
numpy version : 1.19.5
matplotlib version : 3.3.4
pandas version : 1.2.4


In [4]:
## train 데이터 셋 , test 데이터 셋
## train 은 학습을 위한 입력 데이터 셋
## test 은 예측을 위한 새로운 데이터 셋(평가)
## parse_dates : datetime 컬럼을 시간형으로 불러올 수 있음
train = pd.read_csv("bike_mod_tr.csv", parse_dates=['datetime'])
test = pd.read_csv("bike_mod_test.csv", parse_dates=['datetime'])

In [5]:
train.shape, test.shape

((10886, 19), (6493, 16))

In [6]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour,minute,second,dayofweek
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0,0,0,5
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1,0,0,5
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2,0,0,5
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3,0,0,5
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4,0,0,5


In [8]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour,minute,second
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027,2011,1,20,3,0,0,0
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,1,0,0
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000,2011,1,20,3,2,0,0
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,3,0,0
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014,2011,1,20,3,4,0,0


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  int64         
 13  month       10886 non-null  int64         
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  int64         
 16  minute      10886 non-

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6493 entries, 0 to 6492
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    6493 non-null   datetime64[ns]
 1   season      6493 non-null   int64         
 2   holiday     6493 non-null   int64         
 3   workingday  6493 non-null   int64         
 4   weather     6493 non-null   int64         
 5   temp        6493 non-null   float64       
 6   atemp       6493 non-null   float64       
 7   humidity    6493 non-null   int64         
 8   windspeed   6493 non-null   float64       
 9   year        6493 non-null   int64         
 10  month       6493 non-null   int64         
 11  day         6493 non-null   int64         
 12  dayofweek   6493 non-null   int64         
 13  hour        6493 non-null   int64         
 14  minute      6493 non-null   int64         
 15  second      6493 non-null   int64         
dtypes: datetime64[ns](1), fl

## 모델을 위한 데이터 선택

In [11]:
sel = ['hour', 'temp']
# 훈련용
X = train[sel]
y = train['count']
# 평가용
X_val = test[sel]

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

## 딥러닝 구조 결정

* 케라스 라이브러리 중에서 Sequential 함수는 딥러닝의 구조를 한층 한층 쉽게 쌓아올릴 수 있다.
* Sequential() 함수 선언 후, 신경망의 층을 쌓기 위해 model.add() 함수를 사용한다
* input_dim 입력층 노드의 수
* activation - 활성화 함수 선언 (relu, sigmoid)
* Dense() 함수를 이용하여 각 층에 세부 내용을 설정해 준다.

In [14]:
# tensorflow, pytorch, keras
from keras.models import Sequential
from keras.layers import Dense

In [15]:
model = Sequential()
model.add( Dense(16, input_dim=2, activation='relu'))
model.add( Dense(1) )

In [17]:
# model.summary()

## 미니배치의 이해

* 이미지를 하나씩 학습시키는 것보다 여러 개를 한꺼번에 학습시키는 쪽이 효과가 좋다.
* 많은 메모리와 높은 컴퓨터 성능이 필요하므로 일반적으로 데이터를 적당한 크기로 잘라서 학습시킨다.
    * 미니배치라고 한다.

## 딥러닝 실행

In [18]:
# 모델의 사용하는 지표 등..
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
817/817 [==============================] - 1s 674us/step - loss: 48611.3164
Epoch 2/20
817/817 [==============================] - 1s 1ms/step - loss: 24952.2305A: 0s - loss: 25
Epoch 3/20
817/817 [==============================] - 1s 798us/step - loss: 24459.3223
Epoch 4/20
817/817 [==============================] - 1s 678us/step - loss: 24118.2285
Epoch 5/20
817/817 [==============================] - 1s 699us/step - loss: 23717.3848
Epoch 6/20
817/817 [==============================] - 1s 667us/step - loss: 23300.7793
Epoch 7/20
817/817 [==============================] - 1s 763us/step - loss: 22897.5547
Epoch 8/20
817/817 [==============================] - 1s 697us/step - loss: 22479.8750
Epoch 9/20
817/817 [==============================] - 1s 795us/step - loss: 22085.3613
Epoch 10/20
817/817 [==============================] - 1s 727us/step - loss: 21730.9062
Epoch 11/20
817/817 [==============================] - 1s 702us/step - loss: 21425.8555
Epoch 12/20
817/817 [======

In [19]:
# 평가 확인
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 575us/step - loss: 19551.1621


19551.162109375

In [20]:
# 예측하기
pred = model.predict(X_val)
pred[0:10]

array([[ 15.665418],
       [ 30.663118],
       [ 50.8901  ],
       [ 71.119194],
       [ 91.34829 ],
       [110.07057 ],
       [124.491295],
       [121.40751 ],
       [118.32372 ],
       [126.915665]], dtype=float32)

In [22]:
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub_20211102.csv", index=False)

In [39]:
# 특성 2개
# kaggle 점수 : 1.20727

# 모델 개선

* 변수 추가, 성능 개선 확인
* 은닉층 추가
* 노드 수를 늘리기
* epochs 수 변경

### 변수 추가

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 19 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   datetime    10886 non-null  datetime64[ns]
 1   season      10886 non-null  int64         
 2   holiday     10886 non-null  int64         
 3   workingday  10886 non-null  int64         
 4   weather     10886 non-null  int64         
 5   temp        10886 non-null  float64       
 6   atemp       10886 non-null  float64       
 7   humidity    10886 non-null  int64         
 8   windspeed   10886 non-null  float64       
 9   casual      10886 non-null  int64         
 10  registered  10886 non-null  int64         
 11  count       10886 non-null  int64         
 12  year        10886 non-null  int64         
 13  month       10886 non-null  int64         
 14  day         10886 non-null  int64         
 15  hour        10886 non-null  int64         
 16  minute      10886 non-

In [23]:
train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count',
       'year', 'month', 'day', 'hour', 'minute', 'second', 'dayofweek'],
      dtype='object')

In [25]:
sel = ['season', 'holiday', 'workingday', 'weather', 'temp',
       'humidity', 'windspeed', 'year', 'month', 'day', 'hour', 'dayofweek']
X = train[sel]
y = train['count']
X_val = test[sel]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [27]:
model = Sequential()
model.add( Dense(16, input_dim=12, activation='relu') )
model.add( Dense(1) )

In [29]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
817/817 [==============================] - 1s 668us/step - loss: 175799.9844
Epoch 2/20
817/817 [==============================] - 1s 776us/step - loss: 28168.5039
Epoch 3/20
817/817 [==============================] - 1s 693us/step - loss: 25064.2285
Epoch 4/20
817/817 [==============================] - 1s 658us/step - loss: 23205.6816
Epoch 5/20
817/817 [==============================] - 1s 659us/step - loss: 22488.7168
Epoch 6/20
817/817 [==============================] - 1s 662us/step - loss: 22204.3203
Epoch 7/20
817/817 [==============================] - 1s 677us/step - loss: 22078.7285
Epoch 8/20
817/817 [==============================] - 1s 770us/step - loss: 21967.8867
Epoch 9/20
817/817 [==============================] - 1s 682us/step - loss: 21948.5625
Epoch 10/20
817/817 [==============================] - 1s 737us/step - loss: 21869.3027
Epoch 11/20
817/817 [==============================] - 1s 658us/step - loss: 21761.8652
Epoch 12/20
817/817 [===================

In [30]:
# 평가 확인
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 645us/step - loss: 20170.7109


20170.7109375

In [31]:
pred = model.predict(X_val)

In [32]:
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub2_20211102.csv", index=False)

In [44]:
# 특성 12개
# kaggle 점수 : 1.25085

### 변수 변경 2

In [45]:
sel = ['weather', 'temp', 'humidity', 'windspeed', 'year', 'month', 'day', 'hour']

In [46]:
X = train[sel]
y = train['count']
X_val = test[sel]

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [50]:
model = Sequential()
model.add( Dense(16, input_dim=8, activation='relu') )
model.add( Dense(1) )

In [51]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
817/817 [==============================] - 1s 664us/step - loss: 32363.3203
Epoch 2/20
817/817 [==============================] - 1s 825us/step - loss: 29599.4238
Epoch 3/20
817/817 [==============================] - 1s 681us/step - loss: 27526.3848
Epoch 4/20
817/817 [==============================] - 1s 660us/step - loss: 25778.2285
Epoch 5/20
817/817 [==============================] - 1s 671us/step - loss: 24443.5879
Epoch 6/20
817/817 [==============================] - 1s 666us/step - loss: 23600.3164
Epoch 7/20
817/817 [==============================] - 1s 666us/step - loss: 23025.8242
Epoch 8/20
817/817 [==============================] - 1s 677us/step - loss: 22612.7109
Epoch 9/20
817/817 [==============================] - 1s 671us/step - loss: 22444.8008
Epoch 10/20
817/817 [==============================] - 1s 802us/step - loss: 22276.5664
Epoch 11/20
817/817 [==============================] - 1s 698us/step - loss: 22227.1289
Epoch 12/20
817/817 [====================

In [52]:
pred = model.predict(X_val)

In [53]:
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub4_20211102.csv", index=False)

In [ ]:
# 특성 8개
# kaggle 점수 : 1.32218

### 은닉층 추가

In [33]:
model = Sequential()
model.add( Dense(16, input_dim=12, activation='relu') )
model.add( Dense(16, activation='relu'))
model.add( Dense(1) )

In [35]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
817/817 [==============================] - 1s 710us/step - loss: 29747.5059
Epoch 2/20
817/817 [==============================] - 1s 865us/step - loss: 24970.4473 0s - loss: 
Epoch 3/20
817/817 [==============================] - 1s 804us/step - loss: 23191.2539
Epoch 4/20
817/817 [==============================] - 1s 705us/step - loss: 22653.4043
Epoch 5/20
817/817 [==============================] - 1s 720us/step - loss: 22460.3887
Epoch 6/20
817/817 [==============================] - 1s 705us/step - loss: 22301.0098
Epoch 7/20
817/817 [==============================] - 1s 725us/step - loss: 22167.3730
Epoch 8/20
817/817 [==============================] - 1s 714us/step - loss: 21969.6035
Epoch 9/20
817/817 [==============================] - 1s 714us/step - loss: 21947.2773
Epoch 10/20
817/817 [==============================] - 1s 741us/step - loss: 21824.9727
Epoch 11/20
817/817 [==============================] - 1s 786us/step - loss: 21853.9785 0s - loss: 
Epoch 12/20
817/8

In [36]:
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 681us/step - loss: 19986.2695


19986.26953125

In [38]:
pred = model.predict(X_val)
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub3_20211102.csv", index=False)

In [61]:
# 특성 12개
# 층 2개
# kaggle 점수 : 1.22880

### 노드수 추가

In [69]:
sel = ['hour', 'temp']
# 훈련용
X = train[sel]
y = train['count']
# 평가용
X_val = test[sel]

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [71]:
model = Sequential()
model.add( Dense(30, input_dim=2, activation='relu') )
model.add( Dense(1) )

In [72]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=20, batch_size=10)

Epoch 1/20
817/817 [==============================] - 1s 710us/step - loss: 40756.5391
Epoch 2/20
817/817 [==============================] - 1s 770us/step - loss: 24244.4453
Epoch 3/20
817/817 [==============================] - 1s 692us/step - loss: 23700.6699
Epoch 4/20
817/817 [==============================] - 1s 689us/step - loss: 23017.9062
Epoch 5/20
817/817 [==============================] - 1s 699us/step - loss: 22317.4531
Epoch 6/20
817/817 [==============================] - 1s 698us/step - loss: 21653.0488
Epoch 7/20
817/817 [==============================] - 1s 693us/step - loss: 21047.1914
Epoch 8/20
817/817 [==============================] - 1s 682us/step - loss: 20586.6895
Epoch 9/20
817/817 [==============================] - 1s 676us/step - loss: 20239.9062
Epoch 10/20
817/817 [==============================] - 1s 691us/step - loss: 20023.9414
Epoch 11/20
817/817 [==============================] - 1s 683us/step - loss: 19864.9863
Epoch 12/20
817/817 [====================

In [73]:
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 588us/step - loss: 19339.3691


19339.369140625

In [74]:
pred = model.predict(X_val)
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub5_20211102.csv", index=False)

In [60]:
# 특성 2개
# 노드 30개
# kaggle 점수 : 1.30394

### epochs 변경

In [62]:
sel = ['hour', 'temp']
# 훈련용
X = train[sel]
y = train['count']
# 평가용
X_val = test[sel]

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [64]:
model = Sequential()
model.add( Dense(16, input_dim=2, activation='relu') )
model.add( Dense(1) )

In [65]:
model.compile( loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=30, batch_size=10)

Epoch 1/30
817/817 [==============================] - 1s 755us/step - loss: 41498.9102
Epoch 2/30
817/817 [==============================] - 1s 864us/step - loss: 24875.4199
Epoch 3/30
817/817 [==============================] - 1s 720us/step - loss: 24590.5996
Epoch 4/30
817/817 [==============================] - 1s 681us/step - loss: 24327.7949
Epoch 5/30
817/817 [==============================] - 1s 706us/step - loss: 24090.2129
Epoch 6/30
817/817 [==============================] - 1s 687us/step - loss: 23824.1367
Epoch 7/30
817/817 [==============================] - 1s 686us/step - loss: 23509.3652
Epoch 8/30
817/817 [==============================] - 1s 690us/step - loss: 23245.5859
Epoch 9/30
817/817 [==============================] - 1s 709us/step - loss: 22930.7090
Epoch 10/30
817/817 [==============================] - 1s 702us/step - loss: 22640.5078
Epoch 11/30
817/817 [==============================] - 1s 694us/step - loss: 22332.8789
Epoch 12/30
817/817 [====================

In [66]:
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 833us/step - loss: 19384.6133


19384.61328125

In [68]:
pred = model.predict(X_val)
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub6_20211102.csv", index=False)

In [75]:
# 특성 2개
# 은닉층 1개
# 노드 16개
# kaggle 점수 : 1.18369

### 여러가지 옵션 조정

In [99]:
sel = ['humidity', 'temp', 'month', 'day', 'hour']
X = train[sel]
y = train['count']
X_val = test[sel]

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [101]:
model = Sequential()
model.add( Dense(25, input_dim=5, activation='relu') )
model.add( Dense(25, activation='relu') )
model.add( Dense(25, activation='relu') )
model.add( Dense(1) )

In [103]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=50, batch_size=10)

Epoch 1/50
817/817 [==============================] - 1s 770us/step - loss: 15855.0342 0s - loss: 157
Epoch 2/50
817/817 [==============================] - 1s 866us/step - loss: 15709.8232
Epoch 3/50
817/817 [==============================] - 1s 810us/step - loss: 15740.3320
Epoch 4/50
817/817 [==============================] - 1s 793us/step - loss: 15593.4697
Epoch 5/50
817/817 [==============================] - 1s 768us/step - loss: 15636.3799
Epoch 6/50
817/817 [==============================] - 1s 776us/step - loss: 15649.6406
Epoch 7/50
817/817 [==============================] - 1s 782us/step - loss: 15528.6641
Epoch 8/50
817/817 [==============================] - 1s 769us/step - loss: 15492.3330
Epoch 9/50
817/817 [==============================] - 1s 785us/step - loss: 15511.7393
Epoch 10/50
817/817 [==============================] - 1s 796us/step - loss: 15424.5693
Epoch 11/50
817/817 [==============================] - 1s 782us/step - loss: 15394.1904
Epoch 12/50
817/817 [=====

In [104]:
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 657us/step - loss: 14063.3066


14063.306640625

In [105]:
pred = model.predict(X_val)
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub7_20211102.csv", index=False)

In [106]:
# 특성 5개 ['humidity', 'temp', 'month', 'day', 'hour']
# test_size : 기본 0.25
# 은닉층 3개
# 노드 25개
# epochs 50
# kaggle 점수 : 0.93925

In [107]:
# 최두호님 세팅
# 변수 8개
# 은닉층 4개
# 에포크 500
# kaggle 점수 : 0.55745

## 옵션 조정2 

In [109]:
sel = ['humidity', 'temp', 'month', 'day', 'hour']
X = train[sel]
y = train['count']
X_val = test[sel]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [111]:
model = Sequential()
model.add( Dense(25, input_dim=5, activation='relu') )
model.add( Dense(25, activation='relu') )
model.add( Dense(25, activation='relu') )
model.add( Dense(1) )

In [112]:
model.compile(loss='mean_squared_error', optimizer='rmsprop')
model.fit(X_train, y_train, epochs=500, batch_size=10)

Epoch 1/500
817/817 [==============================] - 1s 923us/step - loss: 27949.2969
Epoch 2/500
817/817 [==============================] - 1s 834us/step - loss: 20928.8438
Epoch 3/500
817/817 [==============================] - 1s 802us/step - loss: 19980.8203
Epoch 4/500
817/817 [==============================] - 1s 786us/step - loss: 19051.8574
Epoch 5/500
817/817 [==============================] - 1s 881us/step - loss: 18560.2500
Epoch 6/500
817/817 [==============================] - 1s 795us/step - loss: 18310.6699
Epoch 7/500
817/817 [==============================] - 1s 903us/step - loss: 18193.2695
Epoch 8/500
817/817 [==============================] - 1s 901us/step - loss: 17986.1992
Epoch 9/500
817/817 [==============================] - 1s 831us/step - loss: 17919.4023
Epoch 10/500
817/817 [==============================] - 1s 865us/step - loss: 17867.4121
Epoch 11/500
817/817 [==============================] - 1s 808us/step - loss: 17787.0762
Epoch 12/500
817/817 [========

817/817 [==============================] - 1s 781us/step - loss: 12988.5029
Epoch 94/500
817/817 [==============================] - 1s 766us/step - loss: 13106.8633
Epoch 95/500
817/817 [==============================] - 1s 786us/step - loss: 13054.1201
Epoch 96/500
817/817 [==============================] - 1s 776us/step - loss: 12981.3428
Epoch 97/500
817/817 [==============================] - 1s 760us/step - loss: 13023.7334
Epoch 98/500
817/817 [==============================] - 1s 758us/step - loss: 12920.1797
Epoch 99/500
817/817 [==============================] - 1s 758us/step - loss: 13059.7236
Epoch 100/500
817/817 [==============================] - 1s 779us/step - loss: 13014.3789
Epoch 101/500
817/817 [==============================] - 1s 755us/step - loss: 12952.7725
Epoch 102/500
817/817 [==============================] - 1s 764us/step - loss: 12883.4922
Epoch 103/500
817/817 [==============================] - 1s 769us/step - loss: 12992.7939
Epoch 104/500
817/817 [=======

817/817 [==============================] - 1s 759us/step - loss: 11317.9443
Epoch 184/500
817/817 [==============================] - 1s 764us/step - loss: 11361.6494
Epoch 185/500
817/817 [==============================] - 1s 781us/step - loss: 11449.0879
Epoch 186/500
817/817 [==============================] - 1s 787us/step - loss: 11396.6777
Epoch 187/500
817/817 [==============================] - 1s 817us/step - loss: 11307.8555
Epoch 188/500
817/817 [==============================] - 1s 763us/step - loss: 11388.6074
Epoch 189/500
817/817 [==============================] - 1s 763us/step - loss: 11245.4580
Epoch 190/500
817/817 [==============================] - 1s 785us/step - loss: 11380.0186
Epoch 191/500
817/817 [==============================] - 1s 859us/step - loss: 11304.9160
Epoch 192/500
817/817 [==============================] - 1s 755us/step - loss: 11200.5410
Epoch 193/500
817/817 [==============================] - 1s 828us/step - loss: 11232.9668
Epoch 194/500
817/817 [=

Epoch 274/500
817/817 [==============================] - 1s 791us/step - loss: 10871.4590
Epoch 275/500
817/817 [==============================] - 1s 772us/step - loss: 10906.7197
Epoch 276/500
817/817 [==============================] - 1s 825us/step - loss: 10872.6719
Epoch 277/500
817/817 [==============================] - 1s 864us/step - loss: 10821.1104
Epoch 278/500
817/817 [==============================] - 1s 820us/step - loss: 10831.2725
Epoch 279/500
817/817 [==============================] - 1s 836us/step - loss: 10926.9316
Epoch 280/500
817/817 [==============================] - 1s 803us/step - loss: 10887.8174
Epoch 281/500
817/817 [==============================] - 1s 856us/step - loss: 10854.9512
Epoch 282/500
817/817 [==============================] - 1s 825us/step - loss: 10871.8301
Epoch 283/500
817/817 [==============================] - 1s 797us/step - loss: 10798.6553
Epoch 284/500
817/817 [==============================] - 1s 801us/step - loss: 10712.9414
Epoch 285/

817/817 [==============================] - 1s 788us/step - loss: 10674.4541
Epoch 365/500
817/817 [==============================] - 1s 779us/step - loss: 10690.1650
Epoch 366/500
817/817 [==============================] - 1s 761us/step - loss: 10560.9668
Epoch 367/500
817/817 [==============================] - 1s 760us/step - loss: 10572.7197
Epoch 368/500
817/817 [==============================] - 1s 756us/step - loss: 10428.6035
Epoch 369/500
817/817 [==============================] - 1s 809us/step - loss: 10692.1602
Epoch 370/500
817/817 [==============================] - 1s 765us/step - loss: 10611.1270
Epoch 371/500
817/817 [==============================] - 1s 806us/step - loss: 10655.1963
Epoch 372/500
817/817 [==============================] - 1s 761us/step - loss: 10622.6523
Epoch 373/500
817/817 [==============================] - 1s 803us/step - loss: 10609.0693
Epoch 374/500
817/817 [==============================] - 1s 792us/step - loss: 10626.1904
Epoch 375/500
817/817 [=

817/817 [==============================] - 1s 799us/step - loss: 10591.4375
Epoch 455/500
817/817 [==============================] - 1s 781us/step - loss: 10479.4443
Epoch 456/500
817/817 [==============================] - 1s 914us/step - loss: 10525.6992
Epoch 457/500
817/817 [==============================] - 1s 837us/step - loss: 10512.9473
Epoch 458/500
817/817 [==============================] - 1s 947us/step - loss: 10531.9521
Epoch 459/500
817/817 [==============================] - 1s 1ms/step - loss: 10462.6426
Epoch 460/500
817/817 [==============================] - 1s 1ms/step - loss: 10410.8105
Epoch 461/500
817/817 [==============================] - 1s 1ms/step - loss: 10430.2256
Epoch 462/500
817/817 [==============================] - 1s 783us/step - loss: 10491.9277
Epoch 463/500
817/817 [==============================] - 1s 779us/step - loss: 10439.9150
Epoch 464/500
817/817 [==============================] - 1s 780us/step - loss: 10377.7402
Epoch 465/500
817/817 [=======

In [113]:
model.evaluate(X_test, y_test)

86/86 [==============================] - 0s 669us/step - loss: 12393.7910


12393.791015625

In [114]:
pred = model.predict(X_val)
sub = pd.read_csv('sampleSubmission.csv')
sub['count'] = pred
sub.loc[sub['count']< 0, 'count'] = 0
sub.to_csv("nn_sub8_20211102.csv", index=False)

In [115]:
# 특성 5개 ['humidity', 'temp', 'month', 'day', 'hour']
# test_size : 기본 0.25
# 은닉층 3개
# 노드 25개
# epochs 500
# kaggle 점수 : 1.43081